# Read SEGY Data

This notebook demonstrates how to read and work with SEGY (SEG Y) seismic data files.

## Overview

SEG Y is the standard format for storing seismic reflection data. This notebook will show you how to:

1. Load SEGY files using GeoSuite
2. Access seismic traces and headers
3. Visualize seismic data

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.io.segy_loader import load_segy

print("GeoSuite imported successfully!")

## 1. Load SEGY File

Load a SEGY file using GeoSuite's `load_segy()` function. The function returns seismic traces (amplitude data) and trace headers (metadata).

In [ ]:
# Example: Load a SEGY file
# In practice, replace 'path/to/your/file.sgy' with your actual file path
# traces, headers = load_segy('path/to/your/file.sgy')

print("Note: To load actual SEGY files, use:")
print("  traces, headers = load_segy('path/to/file.sgy')")
print("\nThe function returns:")
print("  - traces: numpy array of shape (n_traces, n_samples)")
print("  - headers: dictionary or DataFrame with trace header information")
print("\nFor demonstration purposes, we'll create synthetic seismic data.")

# Create synthetic seismic data for demonstration
# Simulate realistic seismic traces with some structure
np.random.seed(42)
n_traces = 100
n_samples = 1000
traces = np.random.randn(n_traces, n_samples)

# Add some coherent events to make it more realistic
for i in range(n_traces):
    # Add a few reflectors at different times
    traces[i, 200:250] += 2.0  # Strong reflector
    traces[i, 500:550] += 1.5  # Medium reflector
    traces[i, 800:850] += 1.0  # Weak reflector

# Create synthetic headers
headers = {
    'trace_number': np.arange(1, n_traces + 1),
    'cdp_x': np.linspace(100000, 110000, n_traces),
    'cdp_y': np.linspace(50000, 51000, n_traces),
    'sample_interval': np.ones(n_traces) * 2000,  # microseconds
    'number_of_samples': np.ones(n_traces) * n_samples
}

print(f"\nCreated synthetic data:")
print(f"  Traces: {n_traces}")
print(f"  Samples per trace: {n_samples}")
print(f"  Trace shape: {traces.shape}")

## 2. Access Seismic Data

SEGY files contain seismic traces (amplitude vs time/depth) and trace headers (metadata).

In [ ]:
# Access seismic data
print(f"Seismic data shape: {traces.shape}")
print(f"Number of traces: {traces.shape[0]}")
print(f"Samples per trace: {traces.shape[1]}")
print(f"\nAmplitude range: {traces.min():.3f} to {traces.max():.3f}")
print(f"Mean amplitude: {traces.mean():.3f}")
print(f"Standard deviation: {traces.std():.3f}")

# Access header information
if isinstance(headers, dict):
    print(f"\nTrace headers available:")
    for key, value in headers.items():
        if isinstance(value, np.ndarray):
            print(f"  {key}: array of shape {value.shape}")
        else:
            print(f"  {key}: {value}")
elif hasattr(headers, 'columns'):
    print(f"\nTrace headers (DataFrame):")
    print(f"  Columns: {headers.columns.tolist()}")
    print(f"  Rows: {len(headers)}")

## 3. Visualize Seismic Data

Create a seismic section display (wiggle plot).

In [ ]:
# Display seismic section using GeoSuite's visualization style
# GeoSuite uses matplotlib with signalplot styling for professional plots
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Method 1: Wiggle plot (variable area)
ax1 = axes[0]
skip = max(1, n_traces // 40)  # Show subset of traces for clarity
trace_spacing = 2.0

for i in range(0, n_traces, skip):
    trace = traces[i]
    trace_normalized = trace / np.abs(traces).max() * trace_spacing
    x_center = i * trace_spacing
    x_vals = np.ones(len(trace_normalized)) * x_center
    
    # Plot wiggle
    ax1.plot(x_vals + trace_normalized, range(n_samples), 'k-', linewidth=0.5, alpha=0.7)
    # Fill positive area
    ax1.fill_betweenx(
        range(n_samples), 
        x_vals, 
        x_vals + trace_normalized,
        where=(trace_normalized > 0),
        color='black',
        alpha=0.3
    )

ax1.set_xlabel('Trace Number')
ax1.set_ylabel('Sample Number')
ax1.set_title('Seismic Section - Wiggle Plot (Synthetic Data)')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

# Method 2: Image display (raster plot)
ax2 = axes[1]
im = ax2.imshow(
    traces.T,  # Transpose so time/depth is vertical
    aspect='auto',
    cmap='seismic',
    interpolation='bilinear',
    extent=[0, n_traces, n_samples, 0]
)
ax2.set_xlabel('Trace Number')
ax2.set_ylabel('Sample Number')
ax2.set_title('Seismic Section - Raster Plot (Synthetic Data)')
plt.colorbar(im, ax=ax2, label='Amplitude')

plt.tight_layout()
plt.show()

## 4. Summary

This notebook demonstrated:

-  Loading SEGY files with `load_segy()` (or synthetic data)
-  Accessing seismic traces and headers
-  Visualizing seismic sections

### Next Steps

- Load your own SEGY files from your data directory
- Process traces (filtering, gain correction, etc.)
- Extract horizons and attributes
- Export processed data to other formats